In [55]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [56]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (120, 120)
Sub_Model = 5


In [57]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    print('Loading Data...')
    pickle_in = open(Split_Path + 'X.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Split_Path + 'y.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    # Normalizing the features between [0, 1]
    #X = X / 255.0

    return X, y


In [58]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from keras.utils.np_utils import to_categorical
    model = Sequential()

    model.add(Conv2D(120, (3, 3), padding='same',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    print(model.summary())

    # Compiling the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(X_train, to_categorical(y_train), batch_size=128, epochs=50, verbose=2,
                        validation_data=(X_test, to_categorical(y_test)), shuffle=True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(path + 'Outputs/BRATS/flair_output/Model_' +
               case + '_' + str(count+1) + '.h5')

    return history, model


In [59]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np

    print('Saving Test Accuracy...')

    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')

    for ans in results:
        file.write(str(ans) + ', ')

    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()


In [60]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint

PATH = path + 'Data/BRATS/mod_data1/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/BRATS/flair_output/Results.txt'
workdir = os.listdir(Work_Dir)
if '.DS_Store' in workdir:
    workdir.remove('.DS_Store')
    print('Removed .DS_Store')
# Calling Functions:
for case in tqdm(workdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    # Joining "train" folder with "Cases" path.
    Split_Path = Case_Path + '/train/'
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))

        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0, 1000)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train,
                               y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')
os.system('say "Models are ready"')


Removed .DS_Store


  0%|          | 0/5 [00:00<?, ?it/s]

Case_5
Loading Data...
Shape of X:	 (20694, 120, 120, 1)
Shape of y:	 (20694,)

Case_5_1
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 120, 120, 120)     1200      
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 120, 120, 120)     0         
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 40, 40, 120)      0         
 g2D)                                                            
                                                                 
 dropout_36 (Dropout)        (None, 40, 40, 120)       0         
                                                                 
 conv2d_28 (Conv2D)          (None, 40, 40, 48)        51888     
                                                                 
 leaky_re_lu_37 (LeakyReLU)  (N

  0%|          | 0/5 [00:01<?, ?it/s]


ValueError: in user code:

    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/Users/vitthal/miniforge3/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 201, in __call_